In [2]:
from textblob import TextBlob
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
data = pd.read_csv("training_set1.csv", encoding = "ISO-8859-1")

In [ ]:
def polarity(score):
    if score == 0:
        return 'Negative'
    if score == 4:
        return 'Positive'

In [ ]:
data['score'] = data['score'].apply(lambda x: polarity(x))

In [ ]:
#Storing the scores in list of dictionaries
scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(data['tweet'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['Tweet'][i]))
    compound = analyzer.polarity_scores(data['tweet'][i])["compound"]
    pos = analyzer.polarity_scores(data['tweet'][i])["pos"]
    neu = analyzer.polarity_scores(data['tweet'][i])["neu"]
    neg = analyzer.polarity_scores(data['tweet'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

#Appending the scores into the dataframe for further analysis 
sentiments_score = pd.DataFrame.from_dict(scores)
data = data.join(sentiments_score)



In [ ]:
def text_blob_sentiment(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

In [ ]:
data['text_blob_score'] = data['tweet'].apply(lambda x: text_blob_sentiment(x))

In [ ]:
def detect_polarity(score):
    if score < 0:
        return 'Negative'
    if score > 0:
        return 'Positive'

In [ ]:
data['blob_polarity'] = data['text_blob_score'].apply(lambda x: detect_polarity(x))

In [ ]:
data['vader_polarity'] = data['Compound'].apply(lambda x: detect_polarity(x))

In [ ]:
data = data[data['blob_polarity'].notna()]
data = data[data['vader_polarity'].notna()]

In [ ]:
vader = 0; text_blob = 0

In [ ]:
def counter(row):
    global vader
    global text_blob
    
    if row['score'] == row['blob_polarity']:
        text_blob+=1
    if row['score'] == row['vader_polarity']:
        vader+=1

In [ ]:
for index,row in data.iterrows():
    counter(row)